In [1]:
# Add testing url body text to csv to train a model:


In [2]:
import os, sys, re, time, gc, csv
read_loc = 'dataset/Raw/Read/'
store_loc = 'dataset/Raw/'
save_loc = f'{store_loc}LIWC/'

read_file = 'True' # URL_True, True, Fake

Notebook_start = time.time()

is_URL = False
sufix = 'base'
folders = ['base_body','base_title']
if 'URL' in read_file:
    is_URL = True
    sufix = 'url'
    folders = ['url_body','url_title']
    

from LIWC_Methods import Custom_Methods 
CM = Custom_Methods()

import pandas as pd
import numpy as np


In [3]:
gc.collect()
folders = ['base_body','base_title']
#Delete all TF_dicts:(Only run for the true part):
for folder in folders:
    loc = f'{save_loc}{folder}/'
    print('Delete loc: ',loc)
    for f in os.listdir(loc):
        os.remove(os.path.join(loc, f))

Delete loc:  dataset/Raw/LIWC/base_body/
Delete loc:  dataset/Raw/LIWC/base_title/


In [4]:
print(f'{read_loc}{read_file}.csv')
df_old = pd.read_csv(f'{read_loc}{read_file}.csv',error_bad_lines=False, quotechar='"', thousands=',' ,low_memory=False)

# add id columns if not there
if 'id' not in list(df_old.columns):
    df_old['id'] = [f'{read_file}_{idx}' for idx in df_old.index]
    col_dict = {'text':'body'}
    keep_col = ['id','title','body']
    df_old.rename(columns=col_dict,inplace=True)
    df_old = df_old[keep_col]

if is_URL:    
    idx = list(df_old[df_old.body != ' '].index)
    df_old = df_old.loc[idx]
else:
    idx = list(df_old[df_old.body != ' '].index)
    df_old = df_old.loc[idx]
    idx = list(df_old[df_old.title != ' '].index)
    df_old = df_old.loc[idx]
    
df_old.sort_values(by=['id'],inplace=True)
df_old.reset_index(drop=True,inplace=True)
df_old['news_url'] = 'NA'
df_old['title_url_min'] = ''
df_old['body_url_min'] = ''
df_old['title_url'] = ''
df_old['body_url'] = ''
df_old['title_stem'] = ''
df_old['body_stem'] = ''
df_old['link_cnt'] = 0
df_old['process_state'] = 'Pass'
df_old

dataset/Raw/Read/True.csv


,id,title,body,news_url,title_url_min,body_url_min,title_url,body_url,title_stem,body_stem,link_cnt,process_state
0,True_0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,NA,,,,,,,0,Pass
1,True_1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,NA,,,,,,,0,Pass
2,True_10,Jones certified U.S. Senate winner despite Moo...,(Reuters) - Alabama officials on Thursday cert...,NA,,,,,,,0,Pass
3,True_100,Senator Warren hits out at 'effort to politici...,WASHINGTON (Reuters) - Democratic Senator Eliz...,NA,,,,,,,0,Pass
4,True_1000,No role for Assad in Syria's future: Tillerson,GENEVA (Reuters) - President Bashar al-Assad a...,NA,,,,,,,0,Pass
...,...,...,...,...,...,...,...,...,...,...,...,...
21411,True_9995,Obama says Clinton never jeopardized national ...,WASHINGTON (Reuters) - U.S. President Barack O...,NA,,,,,,,0,Pass
21412,True_9996,U.S. plans to curb tax 'inversions' could hit ...,LONDON (Reuters) - Planned changes that Presid...,NA,,,,,,,0,Pass
21413,True_9997,U.S. Democrat Clinton downplays chance of cont...,WASHINGTON (Reuters) - Democratic front-runner...,NA,,,,,,,0,Pass
21414,True_9998,Boston Globe denounces Trump candidacy in 'fro...,(Reuters) - Headlines screaming “Deportations ...,NA,,,,,,,0,Pass


In [5]:
df_old.shape

(21416, 12)

In [6]:
cnt = 10000
# cnt = len(df_old)
df_old = df_old.sample(n=cnt, random_state=42)
df_old.reset_index(drop=True,inplace=True)
df_old.shape

(10000, 12)

In [7]:
i = 0
start = time.time()
last_start = start
save_text = True
    
end_cnt = len(df_old)
# end_cnt = 100

for idx, row in df_old[i:end_cnt].iterrows():
    if idx != -1:
        # print(idx,end=', ')
        gc.collect()
        if idx%250==0:
            last_start = CM.print_progress(idx,row,start,last_start)
        # try:
        if is_URL:
            row = CM.scrap_page(row)
        else:
            row = CM.clean_text(row)
        if ("page not found" in row.title_url) or ('page unavailable' in row.title_url) or ("scrapper blocked by website" in row.title_url):
            row.process_state = 'Failed'
            save_text = False
            # print(f'failed to clean text for idx:{idx}, ID: {row["id"]}')

        df = row.copy().to_frame().T
        if pd.notnull(df.body_url[idx]):
            if ((len(df.body_url[idx]) > 25) and (save_text == True)): # for LIWC Processing
                # print('Text Saved')
                for part in ['title','body']:
                    col = ['id',f'{part}_url']
                    id_ = df.id[idx]
                    CM.save_LIWC_csv(save_loc,idx,df,part,sufix, id_)
            else:
                # print(f'Text Not Saved for idx:{idx}, ID: {row["id"]}')
                row.process_state = 'Failed'
                save_text = True
        else:
            # print(f'df.body_url is null for idx:{idx}, ID: {row["id"]}')
            row.process_state = 'Failed'
            save_text = True
             
            
#         except:
#             print(f'failed to clean text for idx:{idx}, ID: {row["id"]}')
            
            
        CM.save_file(idx,df,read_file) 

end = time.time()
print(f'Total time in minutes: {int((end-start)/60)}', end=': ')
# row 296 failed

Total time:0  , Cycle time: 0.0, RowID:0 Attempting URL number:True_3605, URL: NA
Total time:1  , Cycle time: 1.4, RowID:250 Attempting URL number:True_13044, URL: NA
Total time:3  , Cycle time: 1.4, RowID:500 Attempting URL number:True_1730, URL: NA
Total time:4  , Cycle time: 1.5, RowID:750 Attempting URL number:True_12144, URL: NA
Total time:6  , Cycle time: 1.5, RowID:1000 Attempting URL number:True_16189, URL: NA
Total time:7  , Cycle time: 1.4, RowID:1250 Attempting URL number:True_3188, URL: NA
Total time:9  , Cycle time: 1.6, RowID:1500 Attempting URL number:True_2390, URL: NA
Total time:10  , Cycle time: 1.6, RowID:1750 Attempting URL number:True_7233, URL: NA
Total time:12  , Cycle time: 1.4, RowID:2000 Attempting URL number:True_11079, URL: NA
Total time:13  , Cycle time: 1.6, RowID:2250 Attempting URL number:True_13955, URL: NA
Total time:15  , Cycle time: 1.5, RowID:2500 Attempting URL number:True_9239, URL: NA
Total time:16  , Cycle time: 1.5, RowID:2750 Attempting URL nu

In [8]:
loc = f'{store_loc}True.csv'
print(loc)
df = pd.read_csv(loc)

df.process_state = ['Pass' if len(str_)> 10 else 'Failed' for str_ in df.body_url]
# save file
df.to_csv(loc,mode='w', header=True, index=False)
df = pd.read_csv(loc)
df.head()

dataset/Raw/True.csv


,id,title,body,news_url,title_url_min,body_url_min,title_url,body_url,title_stem,body_stem,link_cnt,process_state
0,True_3605,White House says no evidence of Trump team col...,"WASHINGTON (Reuters) - The White House, respon...",NaN,white house says no evidence of trump team col...,"the white house, responding to hearings on ca...",white house says no evidence of trump team col...,the white house responding to hearings on capi...,white hous say evid trump team collus russian,white hous respond hear capitol hill contact o...,0,Pass
1,True_12947,"Saudi king, Turkish president discuss regional...",DUBAI (Reuters) - Saudi Arabia s King Salman d...,NaN,"saudi king, turkish president discuss regional...",saudi arabia s king salman discussed the most...,saudi king turkish president discuss regional ...,saudi arabia s king salman discussed the most ...,saudi king turkish presid discuss region devel...,saudi arabia king salman discuss promin develo...,0,Pass
2,True_12586,"U.S.-backed Syrian forces, Iraqi army coordina...",BEIRUT (Reuters) - U.S.-backed militias in Syr...,NaN,"u.s.-backed syrian forces, iraqi army coordina...",u.s.-backed militias in syria said they have ...,u s backed syrian forces iraqi army coordinate...,u s backed militias in syria said they have fo...,u back syrian forc iraqi armi coordin border sdf,u back militia syria said form joint militari ...,0,Pass
3,True_14641,Russia names nine U.S.-backed news outlets lik...,MOSCOW (Reuters) - Russia on Thursday named ni...,NaN,russia names nine u.s.-backed news outlets lik...,russia on thursday named nine u.s. government...,russia names nine u s backed news outlets like...,russia on thursday named nine u s government s...,russia name nine u back news outlet like label...,russia thursday name nine u govern sponsor new...,0,Pass
4,True_19754,Mexican leftist Obrador leads ahead of 2018 el...,MEXICO CITY (Reuters) - Mexican leftist Andres...,NaN,mexican leftist obrador leads ahead of 2018 el...,"mexican leftist andres manuel lopez obrador, ...",mexican leftist obrador leads ahead of electio...,mexican leftist andres manuel lopez obrador wh...,mexican leftist obrador lead ahead elect poll,mexican leftist andr manuel lopez obrador lead...,0,Pass


In [9]:
len(df[df['process_state']=='Pass'])/len(df)

1.0

In [10]:
len(df[df['process_state']=='Pass'])

10000

In [11]:
len(df)

10000

In [12]:
Notebook_end = time.time()
Total_Notebook_Time = round((Notebook_end - Notebook_start)/60.0,)
print(f'This notebook took {Total_Notebook_Time} min to run.')

This notebook took 63 min to run.
